# Module 9: Agent Orchestration

Orchestration is how you coordinate multiple capabilities (LLM calls, web fetching, document parsing, etc.) into coherent workflows.

**This notebook covers:**
1. Orchestration patterns (simple → complex)
2. Popular frameworks and when to use them
3. Building a simple, understandable orchestrator from scratch
4. A practical example: the cover letter agent

**Philosophy:** Start simple. Add complexity only when you understand why you need it.

---
## Part 1: Orchestration Patterns

### Pattern 1: Sequential Pipeline (Simplest)

```
Input → Step 1 → Step 2 → Step 3 → Output
```

Just functions calling functions. No framework needed.

**Pros:** Easy to understand, debug, and modify  
**Cons:** No branching, no error recovery, no parallelism  
**Use when:** Your workflow is linear and predictable

In [ ]:
# Pattern 1: Sequential Pipeline

def step1_fetch(url: str) -> str:
    """Fetch content from URL."""
    # ... fetch logic ...
    return "fetched content"

def step2_analyze(content: str) -> dict:
    """Analyze content with LLM."""
    # ... LLM call ...
    return {"key_points": ["point1", "point2"]}

def step3_generate(analysis: dict) -> str:
    """Generate output document."""
    # ... generation logic ...
    return "final document"

# The "orchestration" is just function composition
def run_pipeline(url: str) -> str:
    content = step1_fetch(url)
    analysis = step2_analyze(content)
    result = step3_generate(analysis)
    return result

print("Pattern 1: Sequential pipeline - just functions!")

### Pattern 2: State Machine

```
         ┌──────────┐
         │  START   │
         └────┬─────┘
              ▼
         ┌──────────┐
    ┌────│ RESEARCH │────┐
    │    └──────────┘    │
    │ (enough)    (need more)
    ▼                    ▼
┌──────────┐       ┌──────────┐
│ ANALYZE  │       │  FETCH   │──┐
└────┬─────┘       └──────────┘  │
     │                    ▲      │
     │                    └──────┘
     ▼
┌──────────┐
│ GENERATE │
└────┬─────┘
     ▼
┌──────────┐
│   DONE   │
└──────────┘
```

**Pros:** Can loop, branch, retry  
**Cons:** More complex, state management  
**Use when:** Workflow has conditions, loops, or needs to recover from errors

In [ ]:
# Pattern 2: State Machine

from enum import Enum, auto
from dataclasses import dataclass, field
from typing import Any

class State(Enum):
    START = auto()
    RESEARCH = auto()
    FETCH = auto()
    ANALYZE = auto()
    GENERATE = auto()
    DONE = auto()
    ERROR = auto()

@dataclass
class WorkflowContext:
    """Shared state across all steps."""
    state: State = State.START
    input_data: dict = field(default_factory=dict)
    research_results: list = field(default_factory=list)
    analysis: dict = field(default_factory=dict)
    output: str = ""
    error: str = ""
    iterations: int = 0
    max_iterations: int = 5

def run_state_machine(ctx: WorkflowContext) -> WorkflowContext:
    """Execute state machine until DONE or ERROR."""
    while ctx.state not in (State.DONE, State.ERROR):
        ctx.iterations += 1
        if ctx.iterations > ctx.max_iterations:
            ctx.state = State.ERROR
            ctx.error = "Max iterations exceeded"
            break
            
        print(f"  State: {ctx.state.name}")
        
        if ctx.state == State.START:
            ctx.state = State.RESEARCH
            
        elif ctx.state == State.RESEARCH:
            # Decide if we need more research
            if len(ctx.research_results) < 2:
                ctx.state = State.FETCH
            else:
                ctx.state = State.ANALYZE
                
        elif ctx.state == State.FETCH:
            # Simulate fetching
            ctx.research_results.append(f"Result {len(ctx.research_results) + 1}")
            ctx.state = State.RESEARCH  # Loop back
            
        elif ctx.state == State.ANALYZE:
            ctx.analysis = {"summary": "analyzed"}
            ctx.state = State.GENERATE
            
        elif ctx.state == State.GENERATE:
            ctx.output = f"Generated from {len(ctx.research_results)} sources"
            ctx.state = State.DONE
    
    return ctx

# Test it
ctx = WorkflowContext()
result = run_state_machine(ctx)
print(f"\nFinal state: {result.state.name}")
print(f"Output: {result.output}")

### Pattern 3: DAG (Directed Acyclic Graph)

```
        ┌─────────┐
        │  Input  │
        └────┬────┘
             │
     ┌───────┼───────┐
     ▼       ▼       ▼
┌────────┐┌────────┐┌────────┐
│Fetch A ││Fetch B ││Fetch C │  ← Parallel!
└───┬────┘└───┬────┘└───┬────┘
    │         │         │
    └─────────┼─────────┘
              ▼
        ┌──────────┐
        │  Merge   │
        └────┬─────┘
             ▼
        ┌──────────┐
        │ Generate │
        └──────────┘
```

**Pros:** Parallel execution, clear dependencies  
**Cons:** More setup, need to handle concurrency  
**Use when:** Steps can run in parallel, you need speed

### Pattern 4: ReAct (Reasoning + Acting)

The LLM itself decides what to do next.

```
Loop:
  1. LLM: "I should search for company info" (Thought)
  2. System: Executes search tool (Action)
  3. LLM sees results (Observation)
  4. LLM: "Now I have enough info to write" (Thought)
  5. System: Executes write tool (Action)
  6. ...
Until: LLM says "DONE"
```

**Pros:** Flexible, can handle unexpected situations  
**Cons:** Unpredictable, can loop forever, expensive (many LLM calls)  
**Use when:** Task is ambiguous, needs dynamic decision-making

---
## Part 2: Popular Frameworks

| Framework | Best For | Complexity | Notes |
|-----------|----------|------------|-------|
| **LangChain** | General-purpose chains | Medium-High | Large ecosystem, can be overwhelming |
| **LlamaIndex** | RAG / document Q&A | Medium | Great for search over documents |
| **CrewAI** | Multi-agent collaboration | Medium | Agents with roles working together |
| **AutoGen** | Conversational agents | Medium | Microsoft, good for chat-based agents |
| **Haystack** | Search pipelines | Medium | Production-focused, well-documented |
| **DSPy** | Prompt optimization | High | Automatically improves prompts |
| **Prefect/Airflow** | Production workflows | High | Not AI-specific, but battle-tested |
| **DIY** | Learning, simple tasks | Low | What we're building here! |

### My Recommendation for Learning:

1. **Start with DIY** (this notebook) — understand the core concepts
2. **Then try LangChain** — it's the most popular, lots of tutorials
3. **Graduate to specialized tools** as needed (CrewAI for multi-agent, LlamaIndex for RAG)

### Why Not Start with a Framework?

Frameworks hide complexity. That's great for productivity, bad for learning. When something breaks, you need to understand what's happening underneath.

---
## Part 3: Building a Simple Orchestrator

Let's build a minimal but useful orchestrator that:
- Defines workflows as a series of steps
- Passes context between steps
- Handles errors gracefully
- Logs what's happening
- Supports human-in-the-loop checkpoints

In [ ]:
import time
from dataclasses import dataclass, field
from typing import Callable, Any, Optional
from enum import Enum
import traceback


class StepStatus(Enum):
    PENDING = "pending"
    RUNNING = "running"
    COMPLETED = "completed"
    FAILED = "failed"
    SKIPPED = "skipped"


@dataclass
class StepResult:
    """Result of executing a single step."""
    name: str
    status: StepStatus
    output: Any = None
    error: Optional[str] = None
    elapsed_time: float = 0.0


@dataclass
class Step:
    """A single step in a workflow."""
    name: str
    func: Callable
    description: str = ""
    checkpoint: bool = False  # Pause for human approval?
    skip_on_error: bool = False  # Continue if this step fails?


@dataclass
class WorkflowResult:
    """Result of running a complete workflow."""
    success: bool
    steps: list[StepResult] = field(default_factory=list)
    context: dict = field(default_factory=dict)
    total_time: float = 0.0
    
    def summary(self) -> str:
        lines = [f"Workflow {'✅ SUCCEEDED' if self.success else '❌ FAILED'}"]
        lines.append(f"Total time: {self.total_time:.2f}s")
        lines.append("\nSteps:")
        for step in self.steps:
            icon = {"completed": "✅", "failed": "❌", "skipped": "⏭️"}.get(step.status.value, "⏳")
            lines.append(f"  {icon} {step.name} ({step.elapsed_time:.2f}s)")
            if step.error:
                lines.append(f"      Error: {step.error}")
        return "\n".join(lines)


class Orchestrator:
    """Simple workflow orchestrator."""
    
    def __init__(self, name: str = "Workflow", verbose: bool = True):
        self.name = name
        self.steps: list[Step] = []
        self.verbose = verbose
    
    def add_step(
        self,
        name: str,
        func: Callable,
        description: str = "",
        checkpoint: bool = False,
        skip_on_error: bool = False,
    ) -> "Orchestrator":
        """Add a step to the workflow. Returns self for chaining."""
        self.steps.append(Step(
            name=name,
            func=func,
            description=description,
            checkpoint=checkpoint,
            skip_on_error=skip_on_error,
        ))
        return self
    
    def run(self, initial_context: dict = None) -> WorkflowResult:
        """Execute all steps in order."""
        context = initial_context or {}
        results = []
        start_time = time.time()
        success = True
        
        if self.verbose:
            print(f"\n{'='*60}")
            print(f"🚀 Starting: {self.name}")
            print(f"{'='*60}")
        
        for step in self.steps:
            step_start = time.time()
            
            if self.verbose:
                print(f"\n▶️  Step: {step.name}")
                if step.description:
                    print(f"   {step.description}")
            
            # Checkpoint: ask for human approval
            if step.checkpoint:
                if not self._checkpoint(step.name, context):
                    results.append(StepResult(
                        name=step.name,
                        status=StepStatus.SKIPPED,
                        elapsed_time=time.time() - step_start,
                    ))
                    continue
            
            try:
                # Execute the step, passing context
                output = step.func(context)
                
                # Step can return updated context or just a value
                if isinstance(output, dict):
                    context.update(output)
                else:
                    context[step.name] = output
                
                results.append(StepResult(
                    name=step.name,
                    status=StepStatus.COMPLETED,
                    output=output,
                    elapsed_time=time.time() - step_start,
                ))
                
                if self.verbose:
                    print(f"   ✅ Completed in {time.time() - step_start:.2f}s")
                    
            except Exception as e:
                error_msg = str(e)
                results.append(StepResult(
                    name=step.name,
                    status=StepStatus.FAILED,
                    error=error_msg,
                    elapsed_time=time.time() - step_start,
                ))
                
                if self.verbose:
                    print(f"   ❌ Failed: {error_msg}")
                
                if not step.skip_on_error:
                    success = False
                    break
        
        total_time = time.time() - start_time
        
        if self.verbose:
            print(f"\n{'='*60}")
            print(f"{'✅ COMPLETED' if success else '❌ FAILED'} in {total_time:.2f}s")
            print(f"{'='*60}")
        
        return WorkflowResult(
            success=success,
            steps=results,
            context=context,
            total_time=total_time,
        )
    
    def _checkpoint(self, step_name: str, context: dict) -> bool:
        """Pause for human approval. Returns True to continue, False to skip."""
        print(f"\n⏸️  CHECKPOINT before '{step_name}'")
        print(f"   Current context keys: {list(context.keys())}")
        response = input("   Continue? [Y/n/show]: ").strip().lower()
        
        if response == 'show':
            import json
            # Show context (truncated)
            for k, v in context.items():
                v_str = str(v)[:200]
                print(f"   {k}: {v_str}")
            return self._checkpoint(step_name, context)  # Ask again
        
        return response != 'n'


print("✅ Orchestrator class defined")

### Test the Orchestrator

In [ ]:
# Define some simple step functions
def fetch_data(ctx):
    """Simulate fetching data."""
    time.sleep(0.5)  # Simulate work
    return {"raw_data": "This is the fetched content about AI and machine learning."}

def analyze_data(ctx):
    """Analyze the fetched data."""
    time.sleep(0.3)
    data = ctx.get("raw_data", "")
    return {"word_count": len(data.split()), "has_ai": "AI" in data}

def generate_summary(ctx):
    """Generate a summary."""
    time.sleep(0.2)
    return {"summary": f"Document has {ctx['word_count']} words. AI mentioned: {ctx['has_ai']}"}

# Build and run the workflow
workflow = Orchestrator("Simple Analysis Pipeline")
workflow.add_step("fetch", fetch_data, "Fetch content from source")
workflow.add_step("analyze", analyze_data, "Analyze the content")
workflow.add_step("summarize", generate_summary, "Generate summary")

result = workflow.run()
print("\n" + result.summary())

### Test with Checkpoints

In [ ]:
# Same workflow but with a checkpoint before generation
workflow_with_checkpoint = Orchestrator("Pipeline with Review")
workflow_with_checkpoint.add_step("fetch", fetch_data, "Fetch content")
workflow_with_checkpoint.add_step("analyze", analyze_data, "Analyze content")
workflow_with_checkpoint.add_step(
    "summarize", 
    generate_summary, 
    "Generate summary",
    checkpoint=True  # Will pause here!
)

result = workflow_with_checkpoint.run()
print("\nFinal summary:", result.context.get("summary"))

---
## Part 4: Cover Letter Agent (Putting It Together)

Let's build the cover letter workflow using real modules.

In [ ]:
import ollama

# We'll simulate some modules for now
# In practice, you'd import from src/

def step_parse_resume(ctx):
    """Parse resume and extract key information."""
    # In practice: use document parser module
    resume_text = ctx.get("resume_text", "")
    
    # Use LLM to extract structured info
    response = ollama.chat(
        model='llama3',
        messages=[{
            'role': 'user',
            'content': f"""Extract key information from this resume. Return as a structured summary:
- Name
- Current/Recent Role
- Key Skills (top 5)
- Years of Experience
- Notable Achievements (top 3)

Resume:
{resume_text}

Structured Summary:"""
        }]
    )
    
    return {"resume_summary": response['message']['content']}


def step_parse_job(ctx):
    """Parse job posting and extract requirements."""
    job_text = ctx.get("job_text", "")
    
    response = ollama.chat(
        model='llama3',
        messages=[{
            'role': 'user',
            'content': f"""Extract key information from this job posting:
- Job Title
- Company Name
- Required Skills
- Nice-to-have Skills
- Key Responsibilities
- Company Culture hints

Job Posting:
{job_text}

Structured Summary:"""
        }]
    )
    
    return {"job_summary": response['message']['content']}


def step_match_analysis(ctx):
    """Analyze how well resume matches job."""
    response = ollama.chat(
        model='llama3',
        messages=[{
            'role': 'user',
            'content': f"""Compare this resume to this job posting.

Resume Summary:
{ctx['resume_summary']}

Job Summary:
{ctx['job_summary']}

Provide:
1. Top 3 strengths that match the job
2. Top 2 potential gaps to address
3. Unique value proposition (what makes this candidate stand out)

Analysis:"""
        }]
    )
    
    return {"match_analysis": response['message']['content']}


def step_generate_letter(ctx):
    """Generate the cover letter."""
    response = ollama.chat(
        model='llama3',
        messages=[{
            'role': 'user',
            'content': f"""Write a professional cover letter based on this analysis.

Resume Summary:
{ctx['resume_summary']}

Job Summary:
{ctx['job_summary']}

Match Analysis:
{ctx['match_analysis']}

Guidelines:
- Professional but personable tone
- Lead with the strongest match
- Address one gap positively (as growth opportunity)
- Show enthusiasm for the specific company
- Keep to 3-4 paragraphs

Cover Letter:"""
        }]
    )
    
    return {"cover_letter": response['message']['content']}


print("✅ Cover letter steps defined")

In [ ]:
# Build the cover letter workflow
cover_letter_workflow = Orchestrator("Cover Letter Generator")

cover_letter_workflow.add_step(
    "parse_resume",
    step_parse_resume,
    "Extract key info from resume"
)

cover_letter_workflow.add_step(
    "parse_job",
    step_parse_job,
    "Extract requirements from job posting"
)

cover_letter_workflow.add_step(
    "match_analysis",
    step_match_analysis,
    "Analyze resume-job fit",
    checkpoint=True  # Review analysis before generating
)

cover_letter_workflow.add_step(
    "generate_letter",
    step_generate_letter,
    "Generate tailored cover letter"
)

print("✅ Workflow built with", len(cover_letter_workflow.steps), "steps")

In [ ]:
# Test with sample data
sample_resume = """
Jane Smith
Senior Software Engineer

Experience:
- 7 years building web applications
- Led team of 5 engineers at TechCorp
- Architected microservices handling 1M requests/day

Skills: Python, JavaScript, AWS, Docker, PostgreSQL, React

Achievements:
- Reduced deployment time by 80% with CI/CD pipeline
- Mentored 3 junior developers to senior level
- Open source contributor to FastAPI
"""

sample_job = """
Staff Engineer - Platform Team
Acme Inc.

We're looking for an experienced engineer to help build our next-generation platform.

Requirements:
- 5+ years of backend development
- Experience with distributed systems
- Strong Python or Go skills
- Kubernetes experience preferred

Nice to have:
- ML/AI experience
- Technical leadership

About us: We're a fast-growing startup building tools for developers.
We value collaboration, learning, and shipping quality software.
"""

# Run the workflow!
result = cover_letter_workflow.run({
    "resume_text": sample_resume,
    "job_text": sample_job,
})

In [ ]:
# Display the generated cover letter
if result.success:
    print("\n" + "="*60)
    print("GENERATED COVER LETTER")
    print("="*60)
    print(result.context.get("cover_letter", "No letter generated"))
else:
    print("Workflow failed:")
    print(result.summary())

---
## Part 5: Next Steps

### Extend the Orchestrator
- Add **parallel execution** for independent steps
- Add **retry logic** for flaky steps
- Add **logging** to file for debugging
- Add **persistence** to resume failed workflows

### Try a Framework
Now that you understand the concepts, try:
```bash
pip install langchain  # Most popular
pip install crewai     # Multi-agent
```

### Build More Workflows
- Research report generator
- Code review assistant
- Meeting notes summarizer

The orchestrator pattern works for all of these!

## Export as Module

In [ ]:
module_code = '''
"""Simple Workflow Orchestrator."""

import time
from dataclasses import dataclass, field
from typing import Callable, Any, Optional
from enum import Enum


class StepStatus(Enum):
    PENDING = "pending"
    RUNNING = "running"
    COMPLETED = "completed"
    FAILED = "failed"
    SKIPPED = "skipped"


@dataclass
class StepResult:
    name: str
    status: StepStatus
    output: Any = None
    error: Optional[str] = None
    elapsed_time: float = 0.0


@dataclass
class Step:
    name: str
    func: Callable
    description: str = ""
    checkpoint: bool = False
    skip_on_error: bool = False


@dataclass
class WorkflowResult:
    success: bool
    steps: list[StepResult] = field(default_factory=list)
    context: dict = field(default_factory=dict)
    total_time: float = 0.0
    
    def summary(self) -> str:
        lines = [f"Workflow {\"✅ SUCCEEDED\" if self.success else \"❌ FAILED\"}"]
        lines.append(f"Total time: {self.total_time:.2f}s")
        for step in self.steps:
            icon = {"completed": "✅", "failed": "❌", "skipped": "⏭️"}.get(step.status.value, "⏳")
            lines.append(f"  {icon} {step.name} ({step.elapsed_time:.2f}s)")
        return "\\n".join(lines)


class Orchestrator:
    def __init__(self, name: str = "Workflow", verbose: bool = True):
        self.name = name
        self.steps: list[Step] = []
        self.verbose = verbose
    
    def add_step(self, name: str, func: Callable, description: str = "",
                 checkpoint: bool = False, skip_on_error: bool = False) -> "Orchestrator":
        self.steps.append(Step(name, func, description, checkpoint, skip_on_error))
        return self
    
    def run(self, initial_context: dict = None) -> WorkflowResult:
        context = initial_context or {}
        results = []
        start_time = time.time()
        success = True
        
        for step in self.steps:
            step_start = time.time()
            if self.verbose:
                print(f"▶️  {step.name}")
            
            try:
                output = step.func(context)
                if isinstance(output, dict):
                    context.update(output)
                else:
                    context[step.name] = output
                results.append(StepResult(step.name, StepStatus.COMPLETED, output, 
                                         elapsed_time=time.time() - step_start))
            except Exception as e:
                results.append(StepResult(step.name, StepStatus.FAILED, error=str(e),
                                         elapsed_time=time.time() - step_start))
                if not step.skip_on_error:
                    success = False
                    break
        
        return WorkflowResult(success, results, context, time.time() - start_time)
'''

with open('/home/developer/projects/sandbox-experiments/src/orchestrator.py', 'w') as f:
    f.write(module_code.strip())

print("✅ Saved to src/orchestrator.py")